### Multi-Tool Calling with MAF and Azure AI Agent Service

In [ ]:
%pip install agent-framework==1.0.0b251001 python-dotenv azure-ai-projects==1.1.0b4 jsonref==1.1.0

### Setting up the Environment

In [ ]:
import os
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential

load_dotenv()
project_endpoint = os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT")
model = os.getenv("AI_FOUNDRY_DEPLOYMENT_NAME")

print("Project Endpoint: ", project_endpoint)
print("Model: ", model)

### Loading the OpenAPI Tool

In [ ]:
from azure.ai.agents.models import OpenApiAnonymousAuthDetails, OpenApiTool
import jsonref

auth = OpenApiAnonymousAuthDetails()

with open("./countries.json", "r") as f:
    openapi_spec = jsonref.loads(f.read())

openapi_tool = OpenApiTool(
    name="CountriesAPI",
    spec = openapi_spec,
    description="Retrieve country information including population and capital city",
    auth = auth
)


### Loading the Code Interpreter Tool

In [ ]:
from azure.ai.agents.models import CodeInterpreterTool

code_interpreter_tool = CodeInterpreterTool()


### Combining Tools into a ToolSet

In [ ]:
from azure.ai.agents.models import ToolSet

tool_set = ToolSet()

tool_set.add(openapi_tool)
tool_set.add(code_interpreter_tool)


### Creating the Agent with Multiple Tools in Azure AI Agent Service 

In [ ]:
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import AzureCliCredential

client  = AIProjectClient(
    endpoint=project_endpoint,
    credential=AzureCliCredential()
)

created_agent = await client.agents.create_agent(
    model = model,
    name = "Multi-Tool Agent",
    instructions = ("You are a helpful assistant that can search for country information "
                "and weather data using APIs. When asked about countries, use the country "
                "API to find information."
                "When asked about code execution, use the code interpreter to run the code."),
    toolset = tool_set
)

print("Created Agent: ", created_agent.id)

### Creating a copy of the agent in Microsoft Agent Framework

In [ ]:
agent = ChatAgent(
    chat_client = AzureAIAgentClient(project_client=client, agent_id=created_agent.id)
)

### Running Different Queries on the Agent

In [ ]:
query = "What is the name and population of the country that uses currency with abbreviation THB at present (use openapi tool)?"
response = await agent.run(query)
print("Agent Response:", response)


In [ ]:
query = "Can you write a Python function to calculate the factorial of a number using recursion and give me the output for number 100"
response = await agent.run(query)
print("Agent Response:", response)